```
Guide to using the codebase to generate the results and plots.

License:
    BSD 3-Clause License

    Copyright (c) 2022, James Gardner.
    All rights reserved except for those for the gwbench code which remain reserved
    by S. Borhanian; the gwbench code is included in this repository for convenience.

    Redistribution and use in source and binary forms, with or without
    modification, are permitted provided that the following conditions are met:

    1. Redistributions of source code must retain the above copyright notice, this
       list of conditions and the following disclaimer.

    2. Redistributions in binary form must reproduce the above copyright notice,
       this list of conditions and the following disclaimer in the documentation
       and/or other materials provided with the distribution.

    3. Neither the name of the copyright holder nor the names of its
       contributors may be used to endorse or promote products derived from
       this software without specific prior written permission.

    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
    AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
    IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
    DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
    FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
    DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
    SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
    CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
    OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
    OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```

# Guide to using the codebase to replicate the results

This notebook explains how to use this codebase on OzStar to generate all the results and plots to compare different networks of gravitational-wave detectors using `gwbench/`.

### Minimal commands to replicate results and plots

In [ ]:
%%bash
python3 generate_injections.py
python3 generate_symbolic_derivatives.py
sbatch slurm_job_scripts/job_calculate_unified_injections.sh
python3 merge_processed_injections_task_files.py
sbatch slurm_job_scripts/job_plot_detection_rate_and_measurement_errors.sh

### Step-by-step explanation

In [ ]:
%%bash
python3 generate_injections.py

1. `$ python3 generate_injections.py`
    - This script generates the raw injection parameters for all science cases considered, generating 1.5 million samples uniformly in redshift for better resolution. The redshift dimension is split into six major bins, each with 250 thousand injections. The raw injections data files for each science case are saved in `data_raw_injections/` and split evenly across 2048 separate files in `data_raw_injections/task_files/`.

In [ ]:
%%bash
python3 generate_symbolic_derivatives.py

2. `$ python3 generate_symbolic_derivatives.py`
    - This script generates and saves the lambdified detector response derivatives for the symbolic waveforms into `lambdified_functions/` for all detector locations considered.
    - This step is independent of Step 1 but both must occur before Step 3.

In [ ]:
%%bash
sbatch slurm_job_scripts/job_calculate_unified_injections.sh

3. `$ sbatch slurm_job_scripts/job_calculate_unified_injections.sh`
    - This slurm script uses memory and time requirements from memory profiling on OzStar's farnarkle 1/2 login node using `$ sbatch slurm_job_scripts/job_memory_profile_unified_injections.sh`. Outputs, both stdout and stderr, from all slurm scripts are saved in `slurm_output_files/`.
    - This job calls `run_calculate_unified_injections_as_task.py` for 2048 primitively parallel tasks. Each task processes one of the raw injection task files in `data_raw_injections/task_files/` uniformly through all of the networks considered using `calculate_unified_injections.py` (built off gwbench's multi-network feature). This is faster than calculating the shared detector responses multiple times and means that the processed data for each network --- which is saved in `data_processed_injections/task_files/` --- represents the same universe.
    - This script also uses the following modules:
        - `filename_search_and_manipulation.py` for input/output from the file system and information in filenames.
        - `networks.py` for the list of considered networks.
        - `network_subclass.py` to load the networks and set relevant attributes.
        - `useful_functions.py` for commonly used but non--implementation specific functions.

In [ ]:
%%bash
python3 merge_processed_injections_task_files.py

4. `$ python3 merge_processed_injections_task_files.py`
    - This script combines the task files in `data_processed_injections/task_files/` into one .npy data file in `data_processed_injections/` for each combination of network and science case, e.g. one file for the `A+_H..A+_L..V+_V..K+_K..A+_I` network and binary neutron-star science case drawn from 1024 task files given two total science cases.

In [ ]:
%%bash
sbatch slurm_job_scripts/job_plot_detection_rate_and_measurement_errors.sh

5. `$ sbatch slurm_job_scripts/job_plot_detection_rate_and_measurement_errors.sh`
    - This slurm script uses memory profiling from `$ sbatch slurm_job_scripts/job_memory_profile_plot_detection_rate_and_measurement_errors.sh`.
    - This job calls `run_plot_collated_detection_rate_and_PDFs_and_CDFs_as_task.py` which runs `plot_collated_detection_rate.py` and `plot_collated_PDFs_and_CDFs.py` for each processed data file in `data_processed_injections/` to create plots in `plots/collated_eff_rate_vs_z/` and `plots/collated_PDFs_and_CDFs_snr_errs_sky-area/`, respectively. To produce the distributions in latter plots, the injections in `data_processed_injections/` that are uniform in redshift are resampled using a phenomenological cosmological distribution to plot what is physical.
    - This script also uses the following modules:
        - `constants.py` for phenomenological values and signal-to-noise ratio thresholds.
        - `cosmological_redshift_resampler.py` to resample the linearly uniformly sampled redshifts using a cosmological model.
        - `merger_and_detection_rates.py` for the cosmological model.
        - `results_class.py` to load the processed data files in `data_processed_injections/` and extract the relevant attributes.
        - `useful_plotting_functions.py` for commonly used plotting functions.
- All other files are non-critical and used for testing, e.g. workshop.ipynb.

#### Troubleshooting:
- Using %%bash magic inside Jupyter does not have access to local bash aliases.